## Libraries, API Key

Don't forget to insert your Google API key

In [1]:
import googlemaps
import numpy as np
import os
import pandas as pd

GOOGLE_API_KEY = ""
gmaps = googlemaps.Client(key = GOOGLE_API_KEY) 

## Constants

Import folder - the one with files like Leuven_aeds.csv (we have to make names more clear in final version)
Export folder - any folder you want

RATIO_EMERGENCY_NORMAL = 0.65 - this means that ambulance is 35% faster than a regular car for the same distance

In [2]:
import_folder_path = "/Users/Jovan/Desktop/MDA_Project/DataSegmented"
export_folder_path = "/Users/Jovan/Desktop/MDA_Project/DistanceMatrices"

cities = ["Antwerpen", "Brugge", "Brussels", "Charleroi", "Gent", "Leuven", "Liege", "Oostende"]

RATIO_EMERGENCY_NORMAL = 0.65

## Functions

In [3]:
# Adjusting coordinates so they suit directions() function

def format_coordinates(latitude, longitude):
    return f"{latitude}, {longitude}"

In [4]:
# Calculates distance matrix for given sets of origin and destination coordinates

def get_distance_matrix(origins, destinations, mode, coef = 1):
    num_origins = len(origins)
    num_destinations = len(destinations)
    result_matrix = np.zeros((num_origins, num_destinations))

    confirmation = input(f"This will initialize {num_origins*num_destinations} calculations. Are you sure? (yes/no): ")
    if confirmation == "yes":
        for i in range(0, num_origins):
            for j in range(0, num_destinations):
                orig = format_coordinates(origins.iloc[i, 0], origins.iloc[i, 1])
                dest = format_coordinates(destinations.iloc[j, 0], destinations.iloc[j, 1])
                print("Calculating pair: [", i, ",", j, "]")
                result_matrix[i][j] = gmaps.directions(orig, dest, mode)[0]['legs'][0]['duration']['value']
                print("Time:", result_matrix[i][j], "\n")

    return(result_matrix * coef)

## Calculating Distance Matrix

In [5]:
for city in cities:
    # Import data on the city
    os.chdir(import_folder_path)
    aeds = pd.read_csv(city + "_aeds.csv")
    cards = pd.read_csv(city + "_cards.csv")
    vehicles = pd.read_csv(city + "_vehicles.csv")
    print("Importing " + city + " successful.")

    # Make 'mandatory' columns for aeds and vehicles
    aeds['public'] = aeds['public'].fillna(0)
    aeds['public'] = ~aeds['public'].astype(bool)
    aeds = aeds.rename(columns={'public': 'mandatory'})
    vehicles['mandatory'] = 1
    print("Made both 'mandatory' columns.\n")

    # Prepare distance matrix
    aeds_coordinates = aeds[['latitude', 'longitude']]
    vehicles_coordinates = vehicles[['latitude', 'longitude']]
    cards_coordinates = cards[['latitude', 'longitude']]

    print("Calculating aeds-cards walking times...")
    aeds_dm = get_distance_matrix(aeds_coordinates, cards_coordinates, "walking")
    print("Calculating vehicles-cards driving times...")
    vehicles_dm = get_distance_matrix(vehicles_coordinates, cards_coordinates, "driving", RATIO_EMERGENCY_NORMAL)

    aeds_dm = np.hstack((aeds_dm, aeds['mandatory'].values.reshape(-1, 1)))
    vehicles_dm = np.hstack((vehicles_dm, vehicles['mandatory'].values.reshape(-1, 1)))
    distance_matrix = np.vstack((aeds_dm, vehicles_dm))

    # Export
    os.chdir(export_folder_path)
    df = pd.DataFrame(distance_matrix)
    df.to_csv(city + "_distance_matrix.csv", index=False)
    print("\n")

Importing Antwerpen successful.
Made both 'mandatory' columns.

Calculating aeds-cards walking times...


This will initialize 2926266 calculations. Are you sure? (yes/no):  no


Calculating vehicles-cards driving times...


This will initialize 72954 calculations. Are you sure? (yes/no):  no




Importing Brugge successful.
Made both 'mandatory' columns.

Calculating aeds-cards walking times...


This will initialize 132825 calculations. Are you sure? (yes/no):  no


Calculating vehicles-cards driving times...


This will initialize 3300 calculations. Are you sure? (yes/no):  yes


Calculating pair: [ 0 , 0 ]
Time: 1198.0 

Calculating pair: [ 0 , 1 ]
Time: 494.0 

Calculating pair: [ 0 , 2 ]
Time: 672.0 

Calculating pair: [ 0 , 3 ]
Time: 932.0 

Calculating pair: [ 0 , 4 ]
Time: 661.0 

Calculating pair: [ 0 , 5 ]
Time: 661.0 

Calculating pair: [ 0 , 6 ]
Time: 667.0 

Calculating pair: [ 0 , 7 ]
Time: 641.0 

Calculating pair: [ 0 , 8 ]
Time: 635.0 

Calculating pair: [ 0 , 9 ]
Time: 872.0 

Calculating pair: [ 0 , 10 ]
Time: 746.0 

Calculating pair: [ 0 , 11 ]
Time: 637.0 

Calculating pair: [ 0 , 12 ]
Time: 874.0 

Calculating pair: [ 0 , 13 ]
Time: 897.0 

Calculating pair: [ 0 , 14 ]
Time: 754.0 

Calculating pair: [ 0 , 15 ]
Time: 846.0 

Calculating pair: [ 0 , 16 ]
Time: 1121.0 

Calculating pair: [ 0 , 17 ]
Time: 938.0 

Calculating pair: [ 0 , 18 ]
Time: 938.0 

Calculating pair: [ 0 , 19 ]
Time: 819.0 

Calculating pair: [ 0 , 20 ]
Time: 794.0 

Calculating pair: [ 0 , 21 ]
Time: 368.0 

Calculating pair: [ 0 , 22 ]
Time: 368.0 

Calculating pair: [

KeyboardInterrupt: 